In [4]:
from astropy.wcs import WCS
from sqlite3 import connect
from astropy import units as u
from astropy.coordinates import SkyCoord, search_around_sky
import pandas as pd
import sqlite3
import numpy as np
from os import path
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import matplotlib.patches as plp
import joblib
import lamy

import Image_module as im


im_path = input("Укажите название снимка или путь к нему:\n")

if not path.isfile(im_path):
    
        raise IOError(f"No such file \"{im_path}\"")
        
img = im.imread(im_path)

        
im_db_path = input("Укажите каталог снимка или путь к нему:\n")

if not path.isfile(im_db_path):
    
        raise IOError(f"No such file \"{im_db_path}\"")

        
cat_db_path = 'catalogs.db' 


cat_name = input("Выберите каталог для кросс-сопоставления: ps/sdss\n")

eps = input("Введите значение epsilon: ")

obs_name = input("Выберите название обсерватории: ")

w = WCS(img.hdr)

field_center = SkyCoord(*w.wcs.crval, unit = [u.deg, u.deg])
region_width = img.width * (abs(w.wcs.cdelt) * u.deg).to(u.arcsec)[0]
region_height = img.height * (abs(w.wcs.cdelt) * u.deg).to(u.arcsec)[1]

con1 = sqlite3.connect(im_db_path)
df1 = pd.read_sql_query("SELECT * from objects", con1)[['ra_deg', 'ra_err', 'dec_deg', 'dec_err', 'mag', 'mag_err']]

df1.rename(columns={df1.columns[0]: 'RA1',
                    df1.columns[1]: 'e_RA1',
                    df1.columns[2]: 'DE1',
                    df1.columns[3]: 'e_DE1',
                    df1.columns[4]: 'r1',
                    df1.columns[5]: 'e_r1'}, inplace=True)

con1.close()

        
con2 = sqlite3.connect(cat_db_path)
df2 = pd.read_sql_query("SELECT * FROM {} WHERE ({} BETWEEN ? AND ?) AND ({} BETWEEN ? AND ?)".format(cat_name, cat_name + '_RA',cat_name + '_DE'),
                        con2,
                        params = [field_center.ra.value - region_width.value/7200 - 30/3600,
                                  field_center.ra.value + region_width.value/7200 + 30/3600,
                                  field_center.dec.value - region_height.value/7200 - 30/3600,
                                  field_center.dec.value + region_height.value/7200 + 30/3600])

if cat_name == 'ps':  
    
    df2.rename(columns={df2.columns[1]: 'RA2',
                        df2.columns[2]: 'e_RA2',
                        df2.columns[3]: 'DE2',
                        df2.columns[4]: 'e_DE2',
                        df2.columns[5]: 'g2',
                        df2.columns[6]: 'e_g2',
                        df2.columns[7]: 'r2',
                        df2.columns[8]: 'e_r2',
                        df2.columns[9]: 'i2',
                        df2.columns[10]: 'e_i2',
                        df2.columns[11]: 'z2',
                        df2.columns[12]: 'e_z2',
                        df2.columns[13]: 'y2',
                        df2.columns[14]: 'e_y2'}, inplace=True)
else:
    pass

if cat_name == 'sdss':
    
    df2.rename(columns={df2.columns[1]: 'RA2',
                        df2.columns[2]: 'e_RA2',
                        df2.columns[3]: 'DE2',
                        df2.columns[4]: 'e_DE2',
                        df2.columns[5]: 'u2',
                        df2.columns[6]: 'e_u2',
                        df2.columns[7]: 'g2',
                        df2.columns[8]: 'e_g2',
                        df2.columns[9]: 'r2',
                        df2.columns[10]: 'e_r2',
                        df2.columns[11]: 'i2',
                        df2.columns[12]: 'e_i2',
                        df2.columns[13]: 'z2',
                        df2.columns[14]: 'e_z2'}, inplace=True)
else:
    pass


coords1 = SkyCoord(df1.iloc[:, 0].values, df1.iloc[:, 2].values, unit = [u.deg]*2)
coords2 = SkyCoord(df2.iloc[:, 1].values, df2.iloc[:, 3].values, unit = [u.deg]*2)

nnbs = SkyCoord.match_to_catalog_sky(coords1, coords2)

ar = nnbs[1].to(u.arcsec).value>float(eps)
arr = 1*ar
noid = df1.iloc[np.where(arr == 1)]

bb = nnbs[0][np.delete(df1.index.values, noid.index.values)] 

c1, c2 = np.unique(bb, return_counts=True) 
dup = c1[c2 > 1]

numbers = []

for i in dup:
    for j in np.argwhere(nnbs[0] == i).tolist():
        numbers.append(j[0])
        
duplicated = df1.iloc[numbers]


coords11 = SkyCoord(duplicated.iloc[:, 0].values, duplicated.iloc[:, 2].values, unit = [u.deg]*2)

res2 = search_around_sky(coords11, coords2, seplimit = 3*u.arcsec)

a1 = duplicated.iloc[res2[0]].copy()
a1['sep'] = res2[2].value*3600

a = df2.iloc[res2[1]]
a2 = a.copy()

a2['sep'] = res2[2].value*3600
preprod = a1.merge(a2, on='sep')

prod = preprod.copy()

prod['position_test'] = prod.apply(lamy.position_test, axis = 1, args = (0.99997, ))
prod['photometry_test'] = prod.apply(lamy.photometry_test, axis = 1, args = (obs_name, ))
prod['tests_sum'] = 1*prod['position_test'] + 1*prod['photometry_test']

ra = prod.groupby(['RA1'], sort=False)['tests_sum'].max()

candidates = df1.loc[df1['RA1'].astype(float).isin(np.array(ra.loc[ra<2].index, dtype=float).tolist())] 
candidates

Укажите название снимка или путь к нему:
test.fits
Укажите каталог снимка или путь к нему:
test.db
Выберите каталог для кросс-сопоставления: ps/sdss
ps
Введите значение epsilon: 2
Выберите название обсерватории: Maidanak


,RA1,e_RA1,DE1,e_DE1,r1,e_r1
619,243.82515147335,0.179439,14.4122964344937,0.180089,22.901880,0.173876
623,243.825649835791,0.179544,14.412493718802,0.180188,22.552554,0.142880
1308,243.905189966057,0.179018,14.4578002953628,0.179682,22.701699,0.135334
1310,243.905286553653,0.179314,14.4572273192979,0.179961,22.470769,0.132855
1435,243.917638261383,0.186168,14.5070266903285,0.186472,22.240629,0.194564
